In [1]:
import math as m
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
#from scipy import linalg as LA

In [2]:
basis_element = {1 : (0,0,1,1), 2 : (0,1,0,1), 3 : (0,1,1,0), 4 : (1,0,0,1), 5 : (1,0,1,0), 6 : (1,1,0,0)}
basis = {}
for i in range(1, len(basis_element.keys())+1):
    for j in range(1, len(basis_element.keys())+1):
        basis.update({(6*(i-1)+j) : basis_element[i]+basis_element[j]})

In [3]:
def des(vector, x): #annihiltion operator
    old_sign = 1
    new_sign = 1
    vector_temp = list(vector)
    for i in range(8-x):
        if vector_temp[i] == 1:
            #print('found')
            #print(i)
            new_sign = old_sign * (-1)
            old_sign = new_sign         
    vector_temp[-x] = 0
    #print(count)
    #if count%2 == 1:
        #sign = -1
    #print(sign)
    return [new_sign, vector_temp]
    
def crt(vector, x): #creation operator
    old_sign = 1
    new_sign = 1
    vector_temp = list(vector)
    for i in range(8-x):
        if vector_temp[i] == 1:
            new_sign = old_sign * (-1)
            old_sign = new_sign
    vector_temp[-x] = 1
    return [new_sign, vector_temp]


def ucc(vector,i,j,k,l): #double excitation term
    vector_temp = list(vector)
    temp1 = des(vector_temp,l)
    #print(temp1)
    temp2 = [des(temp1[1],k)[0] * temp1[0], des(temp1[1],k)[1]] 
    #print(temp2)
    temp3 = [crt(temp2[1],j)[0] * temp2[0], crt(temp2[1],j)[1]]
    #print(temp3)
    temp4 = [crt(temp3[1],i)[0] * temp3[0], crt(temp3[1],i)[1]]
    for index, val in basis.items(): #search by brute force which basis it hops into
        if val == tuple(temp4[1]):
            temp4 = [temp4[0], str(index)]
    return temp4

In [4]:
def occup(vector): #stores the indices of occupied up's
    occuplist = []
    vector_temp = vector
    for i in range(1,5):
        if vector_temp[-i] == 1:
            occuplist.append(i)
    return occuplist

def unoccup(vector): #stores the indices of unoccupied up's
    unoccuplist = []
    vector_temp = vector
    for i in range(1,5):
        if vector_temp[-i] == 0:
            unoccuplist.append(i)
    return unoccuplist

def occdn(vector): #stores the indices of occupied down's
    occdnlist = []
    vector_temp = vector
    for i in range(5,9):
        if vector_temp[-i] == 1:
            occdnlist.append(i)
    return occdnlist

def unoccdn(vector): #stores the indices of unoccupied down's
    unoccdnlist = []
    vector_temp = vector
    for i in range(5,9):
        if vector_temp[-i] == 0:
            unoccdnlist.append(i)
    return unoccdnlist

In [5]:
def row(q):
    vrow = []
    for l in occup(basis[q]):
        for k in occdn(basis[q]):
            for j in unoccup(basis[q]):
                for i in unoccdn(basis[q]):
                    vrow.append(ucc(basis[q],i,j,k,l))
    vrow.append(ucc(basis[q], unoccup(basis[q])[1],unoccup(basis[q])[0],occup(basis[q])[1],occup(basis[q])[0]))
    vrow.append(ucc(basis[q], unoccdn(basis[q])[1],unoccdn(basis[q])[0],occdn(basis[q])[1],occdn(basis[q])[0]))
    return vrow

print(row(1))

[[-1, '15'], [-1, '27'], [-1, '17'], [-1, '29'], [1, '9'], [1, '21'], [1, '11'], [1, '23'], [1, '14'], [1, '26'], [1, '16'], [1, '28'], [-1, '8'], [-1, '20'], [-1, '10'], [-1, '22'], [-1, '6'], [-1, '31']]


In [6]:
hopping_matrix = np.zeros([36,36],dtype=np.int32)

In [7]:
for i in range(1,37):
    for j in range(1,19):
        if int(row(i)[j-1][1]) != 0:
            hopping_matrix[i-1][int(row(i)[j-1][1])-1] = int(row(i)[j-1][0])

In [8]:
print(hopping_matrix[0])

[ 0  0  0  0  0 -1  0 -1  1 -1  1  0  0  1 -1  1 -1  0  0 -1  1 -1  1  0
  0  1 -1  1 -1  0 -1  0  0  0  0  0]


In [9]:
print(hopping_matrix)

[[ 0  0  0  0  0 -1  0 -1  1 -1  1  0  0  1 -1  1 -1  0  0 -1  1 -1  1  0
   0  1 -1  1 -1  0 -1  0  0  0  0  0]
 [ 0  0  0  0 -1  0 -1  0 -1 -1  0  1  1  0  1  1  0 -1 -1  0 -1 -1  0  1
   1  0  1  1  0 -1  0 -1  0  0  0  0]
 [ 0  0  0 -1  0  0  1 -1  0  0 -1  1 -1  1  0  0  1 -1  1 -1  0  0 -1  1
  -1  1  0  0  1 -1  0  0 -1  0  0  0]
 [ 0  0 -1  0  0  0 -1 -1  0  0 -1 -1  1  1  0  0  1  1 -1 -1  0  0 -1 -1
   1  1  0  0  1  1  0  0  0 -1  0  0]
 [ 0 -1  0  0  0  0  1  0 -1 -1  0 -1 -1  0  1  1  0  1  1  0 -1 -1  0 -1
  -1  0  1  1  0  1  0  0  0  0 -1  0]
 [-1  0  0  0  0  0  0  1  1 -1 -1  0  0 -1 -1  1  1  0  0  1  1 -1 -1  0
   0 -1 -1  1  1  0  0  0  0  0  0 -1]
 [ 0 -1  1 -1  1  0  0  0  0  0  0 -1  0 -1  1 -1  1  0  0 -1  1 -1  1  0
  -1  0  0  0  0  0  0  1 -1  1 -1  0]
 [-1  0 -1 -1  0  1  0  0  0  0 -1  0 -1  0 -1 -1  0  1 -1  0 -1 -1  0  1
   0 -1  0  0  0  0  1  0  1  1  0 -1]
 [ 1 -1  0  0 -1  1  0  0  0 -1  0  0  1 -1  0  0 -1  1  1 -1  0  0 -1  1
   0  0 -1  0  0  0 -1

In [661]:
len(hopping_matrix)

36